In [76]:
WORDS = '../Data/words_dense.json'
ALTERNATES = '../Data/alternates.json'

In [77]:
def generate_spelling_alternatives(input_word, word_frequency_dict, letter_alternatives_dict):
    if input_word in word_frequency_dict:
        # If the input word exists in the word frequency dictionary, no correction needed.
        return [input_word]
    elif input_word in letter_alternatives_dict:
        # If the input word has letter alternatives, generate and return word alternatives.
        letter_alternatives = letter_alternatives_dict[input_word]
        word_alternatives = generate_word_alternatives(input_word, letter_alternatives)
        return word_alternatives
    else:
        # Generate and rank candidate words as needed.
        candidate_words = generate_candidates(input_word, word_frequency_dict)
        return candidate_words

In [78]:
from itertools import product

def generate_word_alternatives(input_word, letter_alternatives):
    word_alternatives = set()  # To store the generated word alternatives, using a set to avoid duplicates

    # Base case: If the input word is empty, add it to the alternatives.
    if not input_word:
        word_alternatives.add(input_word)
        # return list(word_alternatives)

    # Recursive case: Get the first character of the input word.
    first_char = input_word[0]

    if first_char in letter_alternatives:
        # If there are letter alternatives for the first character, consider them.
        alt_chars = letter_alternatives[first_char]
    else:
        # If no letter alternatives, use the original character.
        alt_chars = [first_char]

    # Recursively generate alternatives for the rest of the input word.
    rest_alternatives = generate_word_alternatives(input_word[1:], letter_alternatives)

    # Combine the alternative characters with the rest of the alternatives.
    for alt_char, rest_alt in product(alt_chars, rest_alternatives):
        word_alternatives.add(alt_char + rest_alt)

    return list(word_alternatives)

In [79]:
def calculate_levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return calculate_levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 2  # Higher cost for insertions
            deletions = current_row[j] + 2  # Higher cost for deletions
            substitutions = previous_row[j] + (c1 != c2)
            # Include transpositions (swaps)
            if i > 0 and j > 0 and c1 == s2[j - 1] and c2 == s1[i - 1]:
                transpositions = previous_row[j - 1] + 1
                current_row.append(min(insertions, deletions, substitutions, transpositions))
            else:
                current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

In [80]:
def rank_candidates(input_word, word_frequency_dict, max_distance=2, top_n=10):
    candidate_words = []

    for word in word_frequency_dict:
        distance = calculate_levenshtein_distance(input_word, word)
        if distance <= max_distance:
            candidate_words.append((word, distance))

    # Sort candidates by edit distance
    candidate_words.sort(key=lambda x: x[1])

    # Get the top N candidates
    top_candidates = [word for word, _ in candidate_words[:top_n]]

    return top_candidates

In [81]:
import json

# Load the word frequency dictionary from a JSON file
with open(WORDS, 'r') as file:
    word_frequency_dict = json.load(file)

# Load the dictionary of possible letter alternatives from a JSON file
with open(ALTERNATES, 'r') as file:
    letter_alternatives_dict = json.load(file)

In [82]:
# Example usage
input_word = "wrod"  # Example input word

alternatives = generate_spelling_alternatives(input_word, word_frequency_dict, letter_alternatives_dict)
print(alternatives)

['rod', 'wood', 'brad', 'bred', 'bro', 'broad', 'brood', 'bros', 'brow', 'cod']
